# Exercise 1

1. Compute the match-based similarity, cosine similarity and the Jaccard coefficient, between the two sets {A,B,C} and {A,C,D,E}.

2. Compute the Edit distance, and LCSS similarity between (a) ababcabc and babcbc (b) cbacbacba and acbacbacb.

3. Compute the cosine measure using the raw frequencies between the following two sentences: (a) “The sly fox jumped over the lazy dog.” and (b) “The dog jumped at the intruder.”


In [ ]:
# Implementazione di Francesco Di Liberti
from scipy.linalg import norm

a = {'A', 'B', 'C'}
b = {'A', 'C', 'D', 'E'}

union = a.union(b)
print(f'a U b = {union}')
#Match-based
avect = [1 if x in a else 0 for x in union]
bvect = [1 if x in b else 0 for x in union]
print(avect, bvect)

s = 0
for i in range(len(union)):
  if avect[i] == bvect[i] and avect[i]== 1:
    s += 1
print(f'Match-Based Similarity = {s}')

#Cosine similarity
s_cosine = s/(norm(avect)*norm(bvect))
print(f'Cosine Similarity = {s_cosine}')

#Jaccard
jaccard = len(a.intersection(b))/len(union)
print(f'Jaccard Similarity = {jaccard}')

In [ ]:
# Implementazione di Francesco Di Liberti
import numpy as np
def edit(x,y, del_cost, ins_cost, rep_cost):
  m = len(x)+1
  n = len(y) +1
  edit = np.zeros((m, n))
  for i in range(m):
    edit[i][0] = i * del_cost
  for j in range(1, n):
    edit[0][j] = j * ins_cost

  for i in range(1, m):
    for j in range(1, n):
      edit[i, j] = min(edit[i-1, j] + del_cost, edit[i, j-1] + ins_cost, edit[i-1, j-1] + (0 if x[i-1] == y[j-1] else rep_cost))

  return edit[m-1,n-1]

def lcss(x,y):
  m = len(x) + 1
  n = len(y) + 1
  lcss = np.zeros((m,n))

  for i in range(1 ,m):
    for j in range(1, n):
      if(x[i-1] == y[j-1]):
        lcss[i,j] = lcss[i-1, j-1] + 1
      else:
        lcss[i,j] = max(lcss[i-1, j], lcss[i, j-1])

  return lcss[m-1,n-1]

In [ ]:
del_cost = 1;
ins_cost = 1;
rep_cost = 2;

s1 = 'ababcabc'
s2 = 'babcbc'
print(f'edit({s1}, {s2} = {edit(s1, s2, del_cost, ins_cost, rep_cost)})')
print(f'lcss({s1}, {s2} = {lcss(s1, s2)})')

s1 = 'cbacbacba'
s2 = 'acbacbacb'
print(f'edit({s1}, {s2} = {edit(s1, s2, del_cost, ins_cost, rep_cost)})')
print(f'lcss({s1}, {s2} = {lcss(s1, s2)})')


In [ ]:
# Calcolo distanza coseno fra due frasi
# Implementazione di Francesco Di Liberti
from collections import Counter
import math

def calculate_norm(counter):
    return math.sqrt(sum(val**2 for val in counter.values()))

sent_a = 'The sly fox jumped over the lazy dog'
sent_b = 'The dog jumped at the intruder'

#Creo le liste con le singole parole delle due frasi
lista_a = [x.lower() for x in sent_a.split()]
lista_b = [x.lower() for x in sent_b.split()]

#creo un insieme con tutte le parole
all_keys = set(lista_a).union(set(lista_b))

#inizializzo due counter a 0 con tutte le parole presenti nelle 2 frasi
freq_a, freq_b = Counter(), Counter()

#aggiorno la frequenza per ottenere la frequenza delle parole nelle frasi
freq_a.update(lista_a)
freq_b.update(lista_b)

prod_scal = sum(freq_a[key] * freq_b[key] for key in all_keys)

norm_a, norm_b = calculate_norm(freq_a), calculate_norm(freq_b)

cosine_measure = prod_scal / (norm_a * norm_b)
print(cosine_measure)

In [ ]:
freq_a['cat']

# Exercise 2
Write a function to compute the edit distance.

In [ ]:
# fatto nell'esercizio precedente

# Exercise 3
Download the KDD Cup Network Intrusion Data Set for the UCI Machine Learning Repository. Create a data set containing only the categorical attributes. Compute the nearest neighbor for each data point using the (a) Match measure, and (b) Inverse Occurrence Frequency Measure.

http://www.lacascia.it/bd2/_kddcup/task.html

http://www.lacascia.it/bd2/_kddcup/kddcup.names

In [ ]:
import pandas as pd
import numpy as np

labels=list(range(1, 42))+['class']

df = pd.read_csv('http://www.lacascia.it/bd2/_kddcup/kddcup.data_10_percent_corrected.csv', names=labels)

print(df.shape)

In [ ]:
df

In [ ]:
# frequenza delle classi presenti nel dataset
df.iloc[:, 41].value_counts()

In [ ]:
# In base a quanto scritto in http://www.lacascia.it/bd2/_kddcup/kddcup.names
# creo i dataframe coi soli categorici, numerici e classi target
df_cat = df.iloc[:, [1, 2, 3, 11, 20, 21]]
print(df_cat.shape)

In [ ]:
# dataframe coi soli numerici
num = list(set(range(41))-{1, 2, 3, 11, 20, 21})
df_num = df.iloc[:, num]
print(df_num.shape)

In [ ]:
# Estraggo il target e metto ad attack. tutte le classi diverse da normal.
df_target = df.iloc[:, 41].to_numpy(copy=True)
print(df_target.shape)

for i in range(df_target.size):
  if df_target[i]!='normal.':
    df_target[i]='attack.'

In [ ]:
# verifico il nuovo vettore dei target
unique_values, counts = np.unique(df_target, return_counts=True)
for value, count in zip(unique_values, counts):
    print(f"{value} occurs {count} times")

In [ ]:
# Implementazione di Francesco Di Liberti
# Match measure per dati categorici
#La distanza è 0 se sono uguali
def match_measure(x1, x2):
  #match_count = sum(cat1 == cat2 for cat1, cat2 in zip(x1, x2))
  match_count = sum(x1 == x2)
  match_m = 1 - match_count / len(x1)
  return match_m

In [ ]:
# Faccio lo split fra train e test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_cat, df_target, random_state=11)

print(X_train.shape)
print(X_test.shape)

# Prendo solo parte del dataset per motivi di tempo di calcolo
X_train = X_train[:1500]
y_train = y_train[:1500]
X_test = X_test[:500]
y_test = y_test[:500]


In [ ]:
# Calcolo la matrice delle distanze per il train
dist_train = np.zeros((X_train.shape[0], X_train.shape[0]))
for i in range(X_train.shape[0]):
  for j in range(X_train.shape[0]):
    dist_train[i, j] = match_measure(X_train.iloc[i, :], X_train.iloc[j, :])
print(f'dist_train: {dist_train.shape}')

# Calcolo la matrice delle distanze del test dal train
dist_test = np.zeros((X_test.shape[0], X_train.shape[0]))
for i in range(X_test.shape[0]):
  for j in range(X_train.shape[0]):
    dist_test[i, j] = match_measure(X_test.iloc[i, :], X_train.iloc[j, :])
print(f'dist_test: {dist_test.shape}')

# Costruisco il modello e ne valuto le performance
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5, metric='precomputed')
knn.fit(X=dist_train, y=y_train)

print(f'Accuracy: {knn.score(dist_test, y_test):.2%}')

In [ ]:
# inverse occurrence frequency measure
from collections import Counter

# calcolo le frequenze dei simboli nelle diverse features
p = []
for i in range(6):
  p += [Counter(X_train.iloc[:,i])]

def inv_occ_freq_dist(x1, x2):
  m = 0
  for i in range(x1.size):
    p2 = (p[i][x1.iat[i]]/p[i].total())
    if p2==0:
      p2=1
    m += (x1.iat[i] == x2.iat[i])/(p2*p2)
  return 1/(1+m)

In [ ]:
# Calcolo la matrice delle distanze per il train
dist_train = np.zeros((X_train.shape[0], X_train.shape[0]))
for i in range(X_train.shape[0]):
  for j in range(X_train.shape[0]):
    dist_train[i, j] = inv_occ_freq_dist(X_train.iloc[i, :], X_train.iloc[j, :])
print(f'dist_train: {dist_train.shape}')

# Calcolo la matrice delle distanze del test dal train
dist_test = np.zeros((X_test.shape[0], X_train.shape[0]))
for i in range(X_test.shape[0]):
  for j in range(X_train.shape[0]):
    dist_test[i, j] = inv_occ_freq_dist(X_test.iloc[i, :], X_train.iloc[j, :])
print(f'dist_test: {dist_test.shape}')

# Costruisco il modello e ne valuto le performance
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5, metric='precomputed')
knn.fit(X=dist_train, y=y_train)

print(f'Accuracy: {knn.score(dist_test, y_test):.2%}')

In [ ]:
p

# Exercise 4
Repeat Exercise 3 using only the quantitative attributes of the data set, and using the $L_p$ norm for values of p = 1, 2, ∞.

In [ ]:
# Faccio lo split fra train e test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_num, df_target, random_state=11)

print(X_train.shape)
print(X_test.shape)

# Prendo solo parte del dataset per analogia ai dati categorici
X_train = X_train[:3705]
y_train = y_train[:3705]
X_test = X_test[:1235]
y_test = y_test[:1235]

In [ ]:
# Costruisco il modello e ne valuto le performance per p=1
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5, p=1)
knn.fit(X_train, y_train)

print(f'Accuracy (1): {knn.score(X_test, y_test):.2%}')

In [ ]:
# Costruisco il modello e ne valuto le performance per p=2
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5, p=2)
knn.fit(X_train, y_train)

print(f'Accuracy (1): {knn.score(X_test, y_test):.2%}')

In [ ]:
# Costruisco il modello e ne valuto le performance per p=inf
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5, metric='chebyshev')
knn.fit(X_train, y_train)

print(f'Accuracy (inf): {knn.score(X_test, y_test):.2%}')

# Exercise 5
Repeat Exercise 3 using all attributes in the dataset. Use the mixed-attribute function, and different combinations of the categorical and quantitative distance functions of Exercises 3 and 4.


# Exercise 6

IMDB sentiment analysis

In [ ]:
import pandas as pd

labels=['text', 'class']
df = pd.read_csv('http://www.lacascia.it/bd2/movie.csv', names=labels)
df = df.drop(0)
print(df.shape)
df.iloc[:,1].value_counts()

In [ ]:
df